In [1]:
! ls -al

total 32
drwxr-xr-x 4 jupyter jupyter 4096 Jun  8 17:28 .
drwxr-xr-x 3 jupyter jupyter 4096 Jun  8 17:24 ..
drwxr-xr-x 8 jupyter jupyter 4096 Jun  8 17:28 .git
drwxr-xr-x 2 jupyter jupyter 4096 Jun  8 17:28 .ipynb_checkpoints
-rw-r--r-- 1 jupyter jupyter   59 Jun  8 17:23 README.md
-rw-r--r-- 1 jupyter jupyter  783 Jun  8 17:26 bhavani.ipynb
-rw-r--r-- 1 jupyter jupyter  783 Jun  8 17:28 pmuk.ipynb
-rw-r--r-- 1 jupyter jupyter  783 Jun  8 17:25 sharma.ipynb
-rw-r--r-- 1 jupyter jupyter    0 Jun  8 17:28 untitled.txt
